In [1]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# Load data
iris = load_iris()
datasets = train_test_split(iris.data, iris.target,test_size=0.2)
train_data, test_data, train_labels, test_labels = datasets

# Normalize
scaler = StandardScaler()
scaler.fit(train_data)

train_data = scaler.transform(train_data)
test_data = scaler.transform(test_data)

clf = MLPClassifier(solver='sgd', hidden_layer_sizes=(3, 2, 5), max_iter=100, batch_size=2)
clf.fit(train_data, train_labels)   
clf.score(train_data, train_labels)

# Bobot layer
i = 0
for w in clf.coefs_:
    print(f"Layer{i}")
    print(w)
    i+=1

print("Jumlah iterasi")
print(clf.n_iter_)

Layer0
[[-0.41737118  0.17404616 -0.48244752]
 [ 0.3110146   0.77892937 -0.1932483 ]
 [ 0.85463744  0.31707493  0.24566981]
 [-0.21687531  0.68307672  0.08779393]]
Layer1
[[-0.79354462 -0.84734224]
 [-0.77280644 -0.32772938]
 [-1.02717823 -0.94194244]]
Layer2
[[ 0.74111614 -0.18249472 -0.88929958 -0.68658184  0.89363164]
 [-0.73511016 -0.21158881 -0.73578539 -0.02444385 -0.92004767]]
Layer3
[[-0.1051237  -0.04857853  0.09302841]
 [-0.15675989  0.06870962  0.2861304 ]
 [ 0.02274889  0.18529515 -0.81486312]
 [ 0.75246826  0.28494133  0.43414881]
 [-0.49399612 -0.75117137 -0.12587558]]
Jumlah iterasi
42


In [4]:
import numpy as np
from random import seed
from random import random, uniform
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
import graphviz

def linear(x):
    return x


def sigmoid(x):
    return 1/(1+np.exp(-x))


def relu(x):
    return np.maximum(x, 0)


def softmax(x):
    net_h = np.array(x)
    numerator = np.exp(net_h)
    denominator = np.sum(np.exp(x))
    softmax_output = numerator / denominator
    return softmax_output


def linear_derivative(x):
    return 1


def sigmoid_derivative(x):
    s = 1 / (1 + np.exp(-x))
    return s * (1 - s)

# todo


def relu_derivative(x):
    return [0 if (el < 0) else 1 for el in x]

# todo
def softmax_derivative(x):
    return [-(1-el) for el in x]


class Layer:
    def __init__(self, n_input, n_nodes):
        self.weights = []
        self.n_input = n_input
        self.n_nodes = n_nodes
        self.activations = ""
        self.input = []
        self.output = []
        self.error = []
        self.updated_weights = []

    def update_weights_back_propagation(self):
        self.weights = self.updated_weights.copy()
        self.updated_weights = []


class NeuralNetwork:
    def __init__(self, file_name, learning_rate=0.1, err_threshold=0.01, max_iter=100, batch_size=2, dataset=load_iris(), n_input=4, n_output=3):
        # Load iris dataset
        self.dataset = dataset  # dataset
        # self.input = dataset.data  # input
        self.target = dataset.target  # target
        self.target_names = dataset.target_names  # target class name
        self.n_attr = n_input  # n input attribute

        # Neural network
        self.learning_rate = learning_rate
        self.err_threshold = err_threshold
        self.max_iter = max_iter
        self.batch_size = batch_size
        self.layers = []
        self.bias = 1
        self.output = []  # final output from forward propagate

        # Back prop
        self.error_hidden_value = 0
        self.updated_weights = []
        self.error = 999  # current error?
        self.weights = []  # last updated weight
        self.predict = []
        
        with open(file_name, "r") as f:
            line = f.readline().split()
            self.n_layers = len(line)  # how many hidden layers + ouput
            
            for i in range(self.n_layers):
                if i == 0:
                    self.layers.append(Layer(self.n_attr, int(line[i])))
                else:
                    self.layers.append(Layer(int(line[i-1]), int(line[i])))

            for i in range(self.n_layers):
                f.readline()
                for j in range(self.layers[i].n_input + 2):
                    weight = []
                    line = f.readline().strip(" \n").split(" ")
                    for k in range(len(line)):
                        if (j == 0):
                            self.layers[i].activations = str(line[k])
                        else:
                            weight.append(float(line[k]))
                    if j!=0:
                        self.layers[i].weights.append(weight)
                        
    def forward_propagation(self, type):
        for i in range(self.n_layers):
            self.layers[i].input = []
        # The first input layer
        if type == "train":
            self.layers[0].input = self.input
        elif type == "predict":
            self.layers[0].input = self.predict

        # All hidden layers
        for i in range(self.n_layers):
            # add bias
            bias_input = self.bias
            if (i == 0):  # if first hidden layer, convert to array from ndarray and then add bias in the last index
                temp_input = []
                for j in range(len(self.layers[i].input)):
                    input_row = []
                    for k in range(len(self.layers[i].input[j])):
                        input_row.append(self.layers[i].input[j][k])
                    input_row.insert(0, (bias_input))
                    temp_input.append(input_row)
                self.layers[i].input = temp_input
            else:  # if not first layer the immediately add the bias in the last index
                for j in range(len(self.layers[i].input)):
                    self.layers[i].input[j].insert(0, (bias_input))

            # calculate sigma
            self.layers[i].output = np.dot(self.layers[i].input, self.layers[i].weights)
                
            # activation function
            for j in range(len(self.layers[i].output)):
                input_next_layer = []  # temporary list to store the next layer's input
                for k in range(len(self.layers[i].output[j])):
                    x = self.layers[i].output[j][k]
                    result = 0
                    if (self.layers[i].activations.lower() == "linear"):
                        result = format(linear(x))
                    elif (self.layers[i].activations.lower() == "sigmoid"):
                        result = format(sigmoid(x))
                    elif (self.layers[i].activations.lower() == "relu"):
                        result = format(relu(x))
                    elif (self.layers[i].activations.lower() == "softmax"):
                        result = format(softmax(x))
                    else:  # if activation is not linier, relu, sigmoid, or softmax
                        print(
                            f"{self.layers[i].activations}: Invalid activation method!")
                        return

                    # append output, actually layers[i].output == layers[i+1].input
                    self.layers[i].output[j][k] = result
                    # append input for next layer in temporary list (input_next_layer)
                    input_next_layer.append(float(result))

                if (i < self.n_layers - 1): # if there is still the next layer
                    # append input for next layer in layers[i+1].input
                    self.layers[i+1].input.append(input_next_layer)

        # output in the last layer
        self.output = self.layers[-1].output.copy()
        print(f"last output : {self.output}")

    # todo
    def error_output(self):
        # get output layer
        output_layer = self.layers[self.n_layers]
        activation_rule = output_layer.activations.lower()
        output_layer.error = output_layer.output.copy()
        for i in range(len(self.input)):
            expected_target = []
            if (self.target[i] == 0):
                expected_target = [1, 0, 0]
            if (self.target[i] == 1):
                expected_target = [0, 1, 0]
            if (self.target[i] == 2):
                expected_target = [0, 0, 1]
            for j in range(3):
                output_layer.error[i][j] = expected_target[j] - output_layer.error[i][j]
        self.error = np.mean(output_layer.error)

        # calculate error per output node
        for i in range(len(output_layer.error)):
            if (activation_rule == "sigmoid"):
                output_layer.error[i] *= sigmoid_derivative(
                    output_layer.output[i])
            elif (activation_rule == "relu"):
                output_layer.error[i] *= relu_derivative(
                    output_layer.output[i])
            elif (activation_rule == "linear"):
                output_layer.error[i] *= linear_derivative(
                    output_layer.output[i])
            elif (activation_rule == "softmax"):
                output_layer.error[i] *= softmax_derivative(
                    output_layer.output[i])
            
        return

    # todo
    def error_hidden(self):
        #menghitung delta net j = delta k * weight j k
        delta_net_j = 0
        for i in range(self.n_layers-1, -1, -1):
            delta_net_j_array = []
            for j in range(len(self.layers[i+1].error)):
                delta_net_j_data = []
                for k in range(len(self.layers[i+1].weights)-1):
                    # print(self.layers[i+1].error)
                    delta_net_j = np.dot(self.layers[i+1].error[j], np.transpose(self.layers[i+1].weights[k]))
                    delta_net_j_data.append(delta_net_j)
                    

                delta_net_j_array.append(delta_net_j_data)
                
            self.layers[i].error = self.layers[i].output.copy()

            #menghitung error di hidden layer
            for j in range(len(self.layers[i].error)):
                if (self.layers[i].activations == "sigmoid"):
                    self.layers[i].error[j] = np.dot(sigmoid_derivative(
                        self.layers[i].output[j]),  delta_net_j_array[j])
                elif (self.layers[i].activations == "relu"):
                    self.layers[i].error[j] = np.dot(relu_derivative(
                        self.layers[i].output[j]), delta_net_j_array[j])
                elif (self.layers[i].activations == "linear"):
                    self.layers[i].error[j] = np.dot(linear_derivative(
                        self.layers[i].output[j]), delta_net_j_array[j])
        
        return

    # todo
    def update_weights(self, row_input, old_weight, error_term):
        new_weight_list = []

        # new weight for hidden layers
        for i in range(len(row_input)):
            new_weight = old_weight[i] - self.learning_rate * error_term * row_input[i]
            new_weight_list.append(new_weight)
        
        return new_weight_list

    # todo
    def back_propagation(self):
        # output layer
        self.error_output()
        output_layer = self.layers[self.n_layers]
        self.error = np.mean(output_layer.error)
        updated_weights_temp = []

        for i in range(len(output_layer.output)):
            updated_weights_temp.append(self.update_weights(
                output_layer.input[i], output_layer.weights, output_layer.error[i]))
        output_layer.updated_weights = output_layer.weights.copy()
        output_layer.updated_weights = np.mean(
            updated_weights_temp, axis=0)
        

        self.error_hidden()
        for i in range(self.n_layers-1, -1 ,-1):
            self.error_hidden_value = np.mean(self.layers[i].error)

            updated_weights_temp_hidden = []
            for j in range(len(self.layers[i].input)):
                updated_weights_temp_hidden.append(self.update_weights(self.layers[i].input[j], self.layers[i].weights, self.layers[i].error[j]))
            self.layers[i].updated_weights = self.layers[i].weights.copy()
            self.layers[i].updated_weights = np.mean(updated_weights_temp_hidden, axis=0)

        # update all weights
        for layer in self.layers:
            layer.update_weights_back_propagation()
        return

    # todo
    def train(self):
        it = 0
        while ((it < self.max_iter) and (self.err_threshold < self.error)):
            output = []
            error = []
            # random.shuffle(self.datasets.data)
            for i in range(int(len(self.dataset.data)/self.batch_size)):
                idx = i * self.batch_size
                self.input = self.dataset.data[idx:idx+self.batch_size]
                self.forward_propagation(type="train")
                self.back_propagation()
                output.append(self.layers[self.n_layers].output)
                error.append(self.layers[self.n_layers].error)
                
            it += 1

        return

    def set_predict(self, input):
        self.predict = input

    def prediction(self):
        self.forward_propagation(type="predict")
        print(self.output)
        # return self.output
    def check_sanity(self):
        for layer in self.layers:
            print(layer.weights)
    
    def draw_model(self):
        f = graphviz.Digraph('Feed Forward Neural Network', filename="model")
        f.attr('node', shape='circle', width='1.0')
        f.edge_attr.update(arrowhead='vee', arrowsize='2')
        
        for i in range(self.n_layers):
            if i == 0:
                for j in range(len(self.layers[i].weights)): #count weights
                    for k in range(len(self.layers[i].weights[j])): #output node
                        if j==0:
                            f.edge(f"bx{j}", f"h{i+1}_{k}", 
                                   f"{self.layers[i].weights[j][k]:.2f}")
                        else:
                            f.edge(f"x{j}", f"h{i+1}_{k}", 
                                   f"{self.layers[i].weights[j][k]:.2f}")
            else:
                for j in range(len(self.layers[i].weights)): #count weights
                    for k in range(len(self.layers[i].weights[j])): #output node
                        if j==0:
                            f.edge(f"bh{i}", f"h{i+1}_{k}", 
                                   f"{self.layers[i].weights[j][k]:.2f}")
                        else:
                            f.edge(f"h{i}_{j-1}", f"h{i+1}_{k}", 
                                   f"{self.layers[i].weights[j][k]:.2f}")
        
        print(f.source)
#         f.render(directory='model').replace('\\', '/')

    def save_model(self, filename):
        new_file = []
        with open(filename) as file:
            lines = [line.rstrip().split() for line in file]

            new_file.append(lines[0])
            new_file.append(lines[1])
            
            for i in range(self.n_layers):
                new_file.append([self.layers[i].activations])
                for new_weight in self.layers[i].updated_weights:
                    new_file.append(new_weight)
                if i < len(b) - 1:
                    new_file.append('')
                    
            for line in range(len(new_file)):
                str_line = ''
                for i in range(len(new_file[line])):
                    str_line += str(new_file[line][i])
                    if i < len(new_file[line]) - 1:
                        str_line += ' '
                new_file[line] = str_line
                
        new_filename = filename.split(".")[0] + "_updated_weights"
        with open('model/' + new_filename, 'w') as f:
            for line in range(len(new_file)):
                f.write(new_file[line])
                if line < len(new_file) - 1:
                    f.write('\n')

seed(1)

# Normalize data
dataset = load_iris()

nn = NeuralNetwork(file_name="model1.txt",dataset=dataset, batch_size=2)
# nn.forward_propagation()
nn.train()
# nn.set_predict([[1.0, 6.5, 2.0, 3.5]])
# nn.prediction()
# nn.draw_model()
nn.save_model("model1.txt")

# ff = NeuralNetwork("model1.txt")
# ff.forward_propagation("train")


last output : [[4.53978687e-05 9.35762297e-14 9.99954602e-01]
 [4.53978687e-05 9.35762297e-14 9.99954602e-01]]


IndexError: list index out of range